In [165]:
import pandas as pd
import numpy as np
from pycountry_convert import country_name_to_country_alpha3
from pycountry_convert import country_alpha3_to_country_alpha2
from pycountry_convert import country_alpha2_to_country_name
from pycountry_convert import map_countries

In [158]:
entry_visas = pd.read_csv('visa-per-country.csv')[17:]

In [159]:
year_of_interest = '2016'

In [160]:
entry_visas.rename(columns={'Total Study\n(excl. Short-term students)': 'Total Study',
                            'Country of nationality': 'Country'}, inplace=True)

In [172]:
filename = 'map_of_immigration_2005.csv'
filename[:19] + 'work_' + filename[19:]

'map_of_immigration_work_2005.csv'

In [174]:
import os

for filename in os.listdir('data'):
    if len(filename) == 27:
        os.rename('data/' + filename, 'data/' + filename[:19] + 'work_' + filename[19:])

In [167]:
for year_of_interest in [str(year) for year in np.arange(2005,2018)]:
    entry_visas_2016 = pd.DataFrame(columns=["Year", "Total", "Country", "Geographical region"])

    entry_visas_2016['Year'] = entry_visas['Quarter'].apply(lambda year: year[:4])
    entry_visas_2016['Country'] = entry_visas['Country']
    entry_visas_2016['Geographical region'] = entry_visas['Geographical region']

    entry_visas_2016['Total'] = entry_visas['Total Study'].apply(lambda number: ('0' if ('z' in number) else number))
    entry_visas_2016['Total'] = entry_visas_2016['Total'].apply(lambda number: number.replace(',',''))
    entry_visas_2016['Total'] = pd.to_numeric(entry_visas_2016['Total'])

    entry_visas_2016 = entry_visas_2016[entry_visas_2016['Year']==year_of_interest][17:]

    entry_visas_2016.Country.replace(to_replace=conversion_of_country_names, inplace=True)

    entry_visas_2016['country_code'] = entry_visas_2016['Country'].apply(country_to_code)
    entry_visas_2016_refugees = entry_visas_2016[entry_visas_2016['country_code']=='Refugees']

    df = entry_visas_2016[entry_visas_2016['country_code'].apply(len) <= 3]
    grouped = df.groupby(by='Country').sum()
    grouped.reset_index(inplace=True)
    grouped['country_code'] = grouped['Country'].apply(country_to_code)


    grouped['Geographical region'] = grouped['Country'].replace(dictionary_of_regions)

    list_of_country_codes = list(set([country_to_code(country) for country in countries]))

    grouped.set_index('country_code',inplace=True)

    for code in list_of_country_codes:
        if code not in grouped.index:
            country_name = country_alpha2_to_country_name(country_alpha3_to_country_alpha2(code))
            grouped.loc[code] = [country_name,0,'Other']

    grouped.reset_index(inplace=True)

    grouped.to_csv('data/map_of_immigration_study_{}.csv'.format(year_of_interest))

In [132]:
entry_visas_2016['Year'] = entry_visas['Quarter'].apply(lambda year: year[:4])
entry_visas_2016['Country'] = entry_visas['Country']
entry_visas_2016['Geographical region'] = entry_visas['Geographical region']

In [133]:
entry_visas_2016['Total'] = entry_visas['Total Study'].apply(lambda number: ('0' if ('z' in number) else number))

In [134]:
entry_visas_2016['Total'] = entry_visas_2016['Total'].apply(lambda number: number.replace(',',''))

In [135]:
entry_visas_2016['Total'] = pd.to_numeric(entry_visas_2016['Total'])

In [138]:
entry_visas_2016 = entry_visas_2016[entry_visas_2016['Year']=='2016'][17:]

In [147]:
# Converts some of the country names to standard

conversion_of_country_names = {'Korea (North)':'North Korea','Korea (South)':'South Korea',
                               'Occupied Palestinian Territories':'the State of Palestine',
                              'Virgin Islands (British)':'British Virgin Islands',
                              'Virgin Islands (US)':'United States Virgin Islands',
                              'Burma':'Myanmar'}

entry_visas_2016.Country.replace(to_replace=conversion_of_country_names, inplace=True)

In [148]:
countries = map_countries().keys()

In [149]:
def country_to_code(country): 
    try:
        code = country_name_to_country_alpha3(country)
    except KeyError:

        # If there is no exact match try sub-strings
        for country_name in countries:
            if country_name in country:
                code = country_name_to_country_alpha3(country_name)
                break
            elif country in country_name:
                code = country_name_to_country_alpha3(country_name)
                break
        else:
            code = country
        
    return code

In [150]:
entry_visas_2016['country_code'] = entry_visas_2016['Country'].apply(country_to_code)

## Separates refugees and deletes very small entries

In [151]:
entry_visas_2016_refugees = entry_visas_2016[entry_visas_2016['country_code']=='Refugees']

df = entry_visas_2016[entry_visas_2016['country_code'].apply(len) <= 3]
grouped = df.groupby(by='Country').sum()
grouped.reset_index(inplace=True)
grouped['country_code'] = grouped['Country'].apply(country_to_code)

# Creates dictionary of geographical regions

In [153]:
dictionary_of_regions = {item['Country']:item['Geographical region'] for [_,item] in df.iterrows()}
grouped['Geographical region'] = grouped['Country'].replace(dictionary_of_regions)

In [155]:
# Fills the country codes that do not show up in the table with zeros

In [156]:
list_of_country_codes = list(set([country_to_code(country) for country in countries]))

grouped.set_index('country_code',inplace=True)

for code in list_of_country_codes:
    if code not in grouped.index:
        country_name = country_alpha2_to_country_name(country_alpha3_to_country_alpha2(code))
        grouped.loc[code] = [country_name,0,'Other']
        
grouped.reset_index(inplace=True)

In [14]:
grouped.to_csv('data/map_of_immigration_2017.csv')

In [18]:
grouped2 = pd.read_csv('data/map_of_immigration_2017.csv', index_col=0)

In [19]:
grouped2

,country_code,Country,Total,Geographical region
0,AFG,Afghanistan,21,Asia Central
1,ALB,Albania,75,Europe Other
2,DZA,Algeria,106,Africa North
3,ASM,American Samoa,0,Oceania
4,AND,Andorra,3,Europe Other
5,AGO,Angola,37,Africa Sub-Saharan
6,AIA,Anguilla (British),0,Other
7,ATG,Antigua and Barbuda,2,America Central and South
8,ARG,Argentina,385,America Central and South
9,ARM,Armenia,39,Europe Other


## Plotting a map of immigration

In [13]:
# Experimenting with plotly

import plotly
import plotly.plotly as py

from plotly.graph_objs import *

In [422]:
plotly.tools.set_credentials_file(username='Campello', api_key='zFzFwmvrlVOPaKseNrb0')
plotly.tools.set_config_file(world_readable=True, sharing='public')

In [437]:
data = [ dict(
        type = 'choropleth',
        locations = grouped['country_code'],
        z = grouped['Total'],
        text = grouped['Country'] + '<br>' + grouped['Geographical region'],
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            len=0.7,
            title = 'Total work visas')
      ), dict(
        type = 'choropleth',
        locations = ['GBR'],
        z = [1],
        text = ['United Kingdom'],
        colorscale = [[0,"rgb(255, 10, 17)"],[1,"rgb(255, 10, 17)"]],
        showscale=False)]

layout = dict(
    title = 'Map of legal work immigration<br>Source:\
            <a href="https://www.gov.uk/government/publications/immigration-statistics-october-to-december-2016/work#data-tables">\
            Home Office</a>',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.plot(fig, validate=False, filename='d3-world-map' )

'https://plot.ly/~Campello/6'

In [1]:
import pandas as pd

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
df = pd.read_csv('data/map_of_immigration_2017.csv', index_col=0)

In [181]:
import numpy as np

year_and_total = []
for year in np.arange(2005,2017):
    total = pd.read_csv('data/map_of_immigration_study_{}.csv'.format(year))['Total'].sum()
    year_and_total += [[year, total]]
    
pd.DataFrame(year_and_total).to_csv('data/total_year_study.csv', index=0, header=['year','Total'])

In [29]:
sorted_countryes = df.sort_values(by='Total',ascending=False)[:8][['Country','Total']].get_values()

In [41]:
df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            245, 246, 247, 248, 249, 250, 251, 252, 253, 254],
           dtype='int64', length=255)

In [43]:
pd.read_csv('data/total_year.csv', index_col=0)

,Total
year,
2005,243267
2006,249479
2007,205079
2008,184156
2009,155551
2010,160638
2011,149239
2012,145037
2013,154693


In [30]:
sorted_countryes

array([['India', 59180],
       ['Australia', 15336],
       ['United States', 13820],
       ['Philippines', 13119],
       ['Canada', 5801],
       ['New Zealand', 5479],
       ['China', 5339],
       ['Japan', 4920]], dtype=object)

In [26]:
def func(a,b,c):
    d = 1
    print(locals())

In [27]:
func(1,2,3)

{'d': 1, 'c': 3, 'b': 2, 'a': 1}
